In [1]:
# Start Spark environment
import findspark
findspark.init()

In [2]:
# Load Python modules
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from random import random
from operator import add
import time

In [3]:
# Pi calculation using Monte Carlo Method
def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

In [4]:
# Starting Pi Parallel (Spark Cluster with JVM)
start_time = time.time()

#park = SparkSession.builder\
#       .appName("JupyterPiParallel_JVM")\
#       .master("local[*]") \
#       .getOrCreate()

spark = SparkSession.builder \
        .appName("JupyterPiParallel_JVM")\
        .master("spark://192.168.1.20:7077") \
        .config("spark.driver.host", "192.168.1.20") \
        .config("spark.default.parallelism", "54") \
        .config("spark.driver.cores", "3") \
        .config("spark.driver.memory", "6g") \
        .config("spark.driver.maxResultSize", "6g") \
        .config("spark.driver.memoryOverhead", "614m") \
        .config("spark.executor.memory", "1gb") \
        .config("spark.executor.cores", "3") \
        .config("spark.executor.memory", "6g") \
        .config("spark.executor.instances", "6") \
        .config("spark.executor.memoryOverhead", "614m")\
        .getOrCreate()

In [5]:
partitions = 2000
n = 100000 * partitions

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)

print("### Pi is roughly %f ###" % (4.0 * count / n))
print("--- Parallel JVM Execution time: %s seconds ---" % (time.time() - start_time))

### Pi is roughly 3.141400 ###
--- Parallel JVM Execution time: 11.409214496612549 seconds ---


In [6]:
# Stop Spark session
spark.stop()